# Multinomial Logistic Regression - Softmax

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777)

### Simple practice

In [27]:
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

In [28]:
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])

In [29]:
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

In [31]:
# softmax activations
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

In [32]:
# Cross entropy cost
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [33]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [34]:
for step in range(2001):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 200 == 0:
        print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

0 6.32341
200 0.528537
400 0.427366
600 0.349891
800 0.275089
1000 0.234347
1200 0.212375
1400 0.194102
1600 0.17865
1800 0.16541
2000 0.153942


In [37]:
a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
print(a, sess.run(tf.arg_max(a, 1)))

# [각 항목에 대한 확률] [최종선택]

[[  8.27903487e-03   9.91712093e-01   8.83155826e-06]] [1]


In [36]:
b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
print(b, sess.run(tf.arg_max(b, 1)))

[[ 0.81663054  0.16733494  0.01603448]] [0]


###  동물의 각 특성으로 종류 예측

In [ ]:
# https://archive.ics.uci.edu/ml/machine-learning-databases/zoo/zoo.data

In [ ]:
# zoo data : all features boolean
# hair, feathers, eggs, milk, airbone, aquatic, predator, toothed, backbone, 
# breathes, venomous, fins, legs, tail, domestic, catsize
# type : 0 ~ 6

In [3]:
data = np.loadtxt('zoo.csv', delimiter=',', dtype=np.float32)
x_data = data[:, 0:-1]
y_data = data[:, [-1]]

In [4]:
x_data.shape

(101, 16)

In [5]:
y_data.shape

(101, 1)

In [6]:
nb_classes = 7  # 0 ~ 6

In [7]:
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

In [8]:
# one hot
# 0 >> [1,0,0,0,0,0,0]
# 6 >> [0,0,0,0,0,0,1]

In [9]:
Y_one_hot = tf.one_hot(Y, nb_classes)    
Y_one_hot

<tf.Tensor 'one_hot:0' shape=(?, 1, 7) dtype=float32>

In [10]:
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # 한 차원이 늘어나기 때문에 줄여줌.
Y_one_hot

<tf.Tensor 'Reshape:0' shape=(?, 7) dtype=float32>

In [11]:
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

Softmax

In [12]:
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)    # softmax activations

Cross entropy

In [13]:
# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [14]:
# Correct prediction Test model
prediction = tf.arg_max(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# Launch graph

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step:{:5}\tLoss:{:.3f}\tAcc:{:.2%}".format(step, loss, acc))
    
    # Prediction
    pred = sess.run(prediction, feed_dict={X: x_data})
    
    # y_data:(N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

Step:    0	Loss:5.106	Acc:37.62%
Step:  100	Loss:0.800	Acc:79.21%
Step:  200	Loss:0.486	Acc:88.12%
Step:  300	Loss:0.349	Acc:90.10%
Step:  400	Loss:0.272	Acc:94.06%
Step:  500	Loss:0.222	Acc:95.05%
Step:  600	Loss:0.187	Acc:97.03%
Step:  700	Loss:0.161	Acc:97.03%
Step:  800	Loss:0.140	Acc:97.03%
Step:  900	Loss:0.124	Acc:97.03%
Step: 1000	Loss:0.111	Acc:97.03%
Step: 1100	Loss:0.101	Acc:99.01%
Step: 1200	Loss:0.092	Acc:100.00%
Step: 1300	Loss:0.084	Acc:100.00%
Step: 1400	Loss:0.078	Acc:100.00%
Step: 1500	Loss:0.072	Acc:100.00%
Step: 1600	Loss:0.068	Acc:100.00%
Step: 1700	Loss:0.064	Acc:100.00%
Step: 1800	Loss:0.060	Acc:100.00%
Step: 1900	Loss:0.057	Acc:100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 T